In [2]:
import time
from tqdm import tqdm

In [3]:
# 셀리움 import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
# bs4 / 404 해결
import urllib.request
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
import urllib.request

In [5]:
options = webdriver.ChromeOptions()

# 크롬 파일 저장 시 경로 설정
options.add_experimental_option(
	"prefs", {
		"download.default_directory": r"//Users/hwanghyeonjun/Documents/GitHub/data/selenium",
		"download.prompt_for_download": False,
		"download.directory_upgrade": True,
		"safebrowsing.enabled": True
	}
)

# 크롬 시크릿 모드로 실행
options.add_argument('incognito')

# user agent 설정
user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
options.add_argument('user-agent=' + user_agent)

In [6]:
# MACOS 크롬 드라이버
driver = webdriver.Chrome('./driver/chromedriver', options=options)

/var/folders/xz/4cx3hhnx0bsgz2sth0876md80000gn/T/ipykernel_9400/2197993641.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver', options=options)


In [7]:
'''
	start_date : 시작 날짜

  calender_btn : 과거 버전 캘린더 버튼
  calender_new_btn : 최신 버전 페이지 캘린더 버튼

  start_year_input : 과거 버전 캘린더 인풋
  start_year_new_input : 최신 버전 캘린더 인풋

  apply_btn : 과거 버전 적용 버튼
  apply_new_btn : 최신 적용 버튼

  csv_download_btn : 과거 버전 다운로드 버튼
  csv_download_new_btn : 최신 버전 다운로드 버튼

  =======================================================
  임시로 주석 / 홈페이지 모바일로 되있다면 주석 풀고 실행(셀리움 코드 실행 셀에도 주식 풀기)
  bottom_modal : 홍페이지버전이 모바일 시 하단 모달
  bottom_mpdal_btn : 모달 닫기 버튼
'''
start_date = '2020-01-01'

calender_btn = '.DatePickerWrapper_icon-wrap__cwTu_'
calender_new_btn = '#flatDatePickerCanvasHol #datePickerIconWrap'

start_year_input = '.NativeDateInput_root__wbgyP > input'
start_year_new_input = '#startDate'

apply_btn = 'HistoryDatePicker_apply-button__fPr_G'
apply_new_btn = '#applyBtn'

csv_download_btn = '.download-data_download-data__jxNYT > a'
csv_download_new_btn = '#column-content > div.float_lang_base_2.downloadDataWrap > div > a'

bottom_modal = '.download-app-view_download-app-view__4d6Eq'
bottom_mpdal_btn = '.inv-button'

In [8]:
# 주요 환율 리스트
names = ['USD', 'JPY', 'EUR', 'CNY', 'HKD', 'GBP', 'CHF', 'CAD', 'AUD', 'NZD', 'SEK', 'DKK', 'NOK', 'SAR', 'KWD', 'BHD',
         'AED', 'THB', 'SGD', 'IDR', 'INR', 'MYR', 'PKR', 'BDT', 'PHP', 'EGP', 'MXN', 'BND']

In [9]:
# 원자재 리스트
list = ['gold', 'crude-oil', 'silver', 'natural-gas', 'copper', 'us-wheat']

In [10]:
# 테스트 코드들
# url = f'https://kr.investing.com/currencies/jpy-krw-historical-data'
# driver.get(url)
# driver.implicitly_wait(30)
# driver.find_element(By.CSS_SELECTOR, calender_btn).click()
# driver.find_element(By.CSS_SELECTOR, start_year_input).clear()
# driver.implicitly_wait(1)
# driver.find_element(By.CSS_SELECTOR, start_year_input).send_keys(start_date)
# driver.implicitly_wait(5)
# driver.find_element(By.CLASS_NAME, apply_btn).click()
# time.sleep(10)
# driver.find_element(By.CSS_SELECTOR, csv_download_btn).click()
# driver.implicitly_wait(30)

In [8]:
# 과거 데이터 크롤링 함수
'''
	인베스팅 과거 버전 페이지 크롤링 함수

'''

def investing_crawling(middel_url, names, start_date, suffix=''):
	suffix_ = suffix
	print('전체 개수 {0}개'.format(len(names)))

	succeed = []
	failed = []

	# 셀리움 실행 코드
	for name in tqdm(names):
		name_ = name.lower()
		try:
			url = f'https://kr.investing.com/{middel_url}/{name_}{suffix_}-historical-data'

			driver.get(url)
			headers = {'User-Agent': 'Chrome/107.0.5304.87'}
			req = urllib.request.Request(url, headers=headers)
			urlopen(req)
			time.sleep(5)

			# 모바일 화면시 주석 풀고 실행
			# is_displayed_modal = driver.find_element(By.CSS_SELECTOR, bottom_modal).is_displayed()
			# if is_displayed_modal:
			# 	driver.find_element(By.CSS_SELECTOR, bottom_mpdal_btn).click()
			# 	driver.implicitly_wait(10)

			driver.find_element(By.CSS_SELECTOR, calender_btn).click()
			driver.find_element(By.CSS_SELECTOR, start_year_input).clear()
			driver.implicitly_wait(1)
			driver.find_element(By.CSS_SELECTOR, start_year_input).send_keys(start_date)
			driver.implicitly_wait(5)
			driver.find_element(By.CLASS_NAME, apply_btn).click()
			time.sleep(3)
			driver.find_element(By.CSS_SELECTOR, csv_download_btn).click()
			driver.implicitly_wait(30)
			succeed.append(name_)
		except HTTPError as e:
			failed.append(name_)

	print('=' * 150)
	print('성공 :{0}\nerror 개수: {1}개'.format(succeed, len(succeed)))
	print('=' * 150)
	print('실패 :{0}\nerror 개수: {1}개'.format(failed, len(failed)))
	print('=' * 150)

In [20]:
# 과거 데이터 크롤링 함수
'''
	인베스팅 최신 버전 페이지 크롤링 함수
'''


def investing_crawling_new(middel_url, names, start_date, suffix=''):
	suffix_ = suffix
	print('전체 개수 {0}개'.format(len(names)))

	succeed = []
	failed = []

	# 셀리움 실행 코드
	for name in tqdm(names):
		name_ = name.lower()
		try:
			url = f'https://kr.investing.com/{middel_url}/{name_}{suffix_}-historical-data'

			driver.get(url)
			headers = {'User-Agent': 'Chrome/107.0.5304.87'}
			req = urllib.request.Request(url, headers=headers)
			urlopen(req)
			time.sleep(5)

			# 모바일 화면시 주석 풀고 실행
			# is_displayed_modal = driver.find_element(By.CSS_SELECTOR, bottom_modal).is_displayed()
			# if is_displayed_modal:
			# 	driver.find_element(By.CSS_SELECTOR, bottom_mpdal_btn).click()
			# 	driver.implicitly_wait(10)

			driver.find_element(By.CSS_SELECTOR, calender_new_btn).click()
			driver.find_element(By.CSS_SELECTOR, start_year_new_input).clear()
			driver.implicitly_wait(1)
			driver.find_element(By.CSS_SELECTOR, start_year_new_input).send_keys(start_date)
			driver.implicitly_wait(5)
			driver.find_element(By.CSS_SELECTOR, apply_new_btn).click()
			time.sleep(3)
			driver.find_element(By.CSS_SELECTOR, csv_download_new_btn).click()
			driver.implicitly_wait(30)
			succeed.append(name_)
		except HTTPError as e:
			failed.append(name_)

	print('=' * 150)
	print('성공 :{0}\n성공 개수: {1}개'.format(succeed, len(succeed)))
	print('=' * 150)
	print('실패 :{0}\nerror 개수: {1}개'.format(failed, len(failed)))
	print('=' * 150)

In [13]:
investing_crawling('currencies', names, '2020-01-01', '-krw')

전체 개수 28개


100%|██████████| 28/28 [04:02<00:00,  8.66s/it]

성공 :['usd', 'jpy', 'eur', 'cny', 'hkd', 'gbp', 'chf', 'cad', 'aud', 'nzd', 'sek', 'dkk', 'nok', 'sar', 'aed', 'thb', 'sgd', 'idr', 'inr', 'myr', 'pkr', 'php', 'mxn']
error 개수: 23개
실패 :['kwd', 'bhd', 'bdt', 'egp', 'bnd']
error 개수: 5개


In [14]:
investing_crawling('commodities', list, start_date)

전체 개수 6개


100%|██████████| 6/6 [00:55<00:00,  9.22s/it]

성공 :['gold', 'crude-oil', 'silver', 'natural-gas', 'copper', 'us-wheat']
error 개수: 6개
실패 :[]
error 개수: 0개


In [102]:
# 드라이버 종료
driver.close()

In [50]:
# 세계지수 리스트 크롤링
url = 'https://kr.investing.com/indices/major-indices'

In [51]:
try:
	headers = {'User-Agent': 'Chrome/107.0.5304.87'}
	req = urllib.request.Request(url, headers=headers)
	html = urlopen(req)
except HTTPError as e:
	err = e.read()
	code = e.getcode()
	print(err, code)

In [52]:
soup = BeautifulSoup(html, 'html.parser')

In [53]:
stock_link = soup.select('td.datatable_cell__0y0eu.datatable_cell--name__5g25Q.table-browser_col-name__qzN_9 > div > a')
stock_link

[<a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/kospi" title="코스피지수">코스피지수</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/kospi-50" title="코스피 50">코스피 50</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/kosdaq" title="코스닥">코스닥</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/us-30" title="다우존스">다우존스</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/us-spx-500" title="S&amp;P 500 - (CFD)">S&amp;P 500</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/nasdaq-composite" title="나스닥종합지수">나스닥종합지수</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/smallcap-2000" title="러셀 2000 지수 - (CFD)">러셀 2000</a>,
 <a class="inv-link bold datatable_cell--name__link__tmnQz" href="/indices/volatility-s-p-500" title="CBOE Volatility Index - (CFD)">CBOE VIX</a>,
 <a class="inv-link bold datatable_cell--

In [54]:
indices = []
for each in stock_link:
	href = each['href']
	indice = href.split('/')[-1]
	indices.append(indice)

In [55]:
print(indices)

['kospi', 'kospi-50', 'kosdaq', 'us-30', 'us-spx-500', 'nasdaq-composite', 'smallcap-2000', 'volatility-s-p-500', 's-p-tsx-composite', 'bovespa', 'ipc', 'germany-30', 'uk-100', 'france-40', 'eu-stoxx50', 'netherlands-25', 'spain-35', 'it-mib-40', 'switzerland-20', 'psi-20', 'bel-20', 'atx', 'omx-stockholm-30', 'mcx', 'rtsi', 'wig-20', 'hungary-stock-market', 'ise-100', 'ta25', 'tasi', 'japan-ni225', 'aus-200', 'shanghai-composite', 'szse-component', 'ftse-china-a50', 'dj-shanghai', 'hang-sen-40', 'taiwan-weighted', 'thailand-set', 'idx-composite', 's-p-cnx-nifty', 'sensex', 'vn-30']


In [ ]:
investing_crawling('indices', indices, '2020-01-01')

In [10]:
# 암호화폐 크롤링
coins = [['bitcoin', 'btc-krw'], ['ethereum', 'eth-krw'], ['xrp', 'xrp-krw']]

In [16]:
for coin in tqdm(coins):
	try:
		url = f'https://kr.investing.com/crypto/{coin[0]}/{coin[1]}-historical-data'

		driver.get(url)
		headers = {'User-Agent': 'Chrome/107.0.5304.87'}
		req = urllib.request.Request(url, headers=headers)
		urlopen(req)
		time.sleep(5)

		# is_displayed_modal = driver.find_element(By.CSS_SELECTOR, bottom_modal).is_displayed()
		# if is_displayed_modal:
		# 	driver.find_element(By.CSS_SELECTOR, bottom_mpdal_btn).click()
		# 	driver.implicitly_wait(10)
		# else:
		# 	pass

		driver.find_element(By.CSS_SELECTOR, calender_btn).click()
		driver.find_element(By.CSS_SELECTOR, start_year_input).clear()
		driver.implicitly_wait(1)
		driver.find_element(By.CSS_SELECTOR, start_year_input).send_keys(start_date)
		driver.implicitly_wait(5)
		driver.find_element(By.CLASS_NAME, apply_btn).click()
		time.sleep(3)
		driver.find_element(By.CSS_SELECTOR, csv_download_btn).click()
		driver.implicitly_wait(30)
	except HTTPError as e:
		pass

100%|██████████| 3/3 [00:34<00:00, 11.61s/it]


In [149]:
driver.close()

SyntaxError: invalid syntax (3245650350.py, line 1)

In [11]:
# 한국, 중국, 일본, 미국 채권
url = 'https://kr.investing.com/rates-bonds/world-government-bonds'

In [17]:
driver.get(url)

In [12]:
try:
	headers = {'User-Agent': 'Chrome/107.0.5304.87'}
	req = urllib.request.Request(url, headers=headers)
	html = urlopen(req)
except HTTPError as e:
	err = e.read()
	code = e.getcode()
	print(err, code)

soup = BeautifulSoup(html, 'html.parser')

In [13]:
rate_bonds_id = ['60', '1', '20', '54']
rate_bonds_link = []

In [14]:
for id_ in rate_bonds_id:
	rate_bond = soup.select(f'#rates_bonds_table_{id_} td.bold.left.noWrap.elp.plusIconTd > a')
	time.sleep(0.5)
	rate_bonds_link.extend(rate_bond)


In [15]:
rate_bonds_link

[<a href="/rates-bonds/south-korea-1-year-bond-yield" title="한국 1년">한국 1년</a>,
 <a href="/rates-bonds/south-korea-2-year-bond-yield" title="한국 2년">한국 2년</a>,
 <a href="/rates-bonds/south-korea-3-year-bond-yield" title="한국 3년">한국 3년</a>,
 <a href="/rates-bonds/south-korea-4-year-bond-yield" title="한국 4년">한국 4년</a>,
 <a href="/rates-bonds/south-korea-5-year-bond-yield" title="한국 5년">한국 5년</a>,
 <a href="/rates-bonds/south-korea-10-year-bond-yield" title="한국 10년">한국 10년</a>,
 <a href="/rates-bonds/south-korea-20-year-bond-yield" title="한국 20년">한국 20년</a>,
 <a href="/rates-bonds/south-korea-30-year" title="한국 30년">한국 30년</a>,
 <a href="/rates-bonds/south-korea-50-year" title="한국 50년">한국 50년</a>,
 <a href="/rates-bonds/u.s.-1-month-bond-yield" title="미국 1개월">미국 1개월</a>,
 <a href="/rates-bonds/u.s.-3-month-bond-yield" title="미국 3개월">미국 3개월</a>,
 <a href="/rates-bonds/u.s.-6-month-bond-yield" title="미국 6개월">미국 6개월</a>,
 <a href="/rates-bonds/u.s.-1-year-bond-yield" title="미국 1년">미국 1년</a>,
 <

In [16]:
rate_bonds = []
for each in rate_bonds_link:
	href = each['href']
	rate_bond = href.split('/')[-1]

	rate_bonds.append(rate_bond)

	middel_url = href.split('/')[-2]


In [19]:
investing_crawling_new(middel_url, rate_bonds, start_date)

전체 개수 47개


100%|██████████| 47/47 [13:51<00:00, 17.69s/it]

성공 :['south-korea-1-year-bond-yield', 'south-korea-2-year-bond-yield', 'south-korea-3-year-bond-yield', 'south-korea-4-year-bond-yield', 'south-korea-5-year-bond-yield', 'south-korea-10-year-bond-yield', 'south-korea-20-year-bond-yield', 'south-korea-30-year', 'south-korea-50-year', 'u.s.-1-month-bond-yield', 'u.s.-3-month-bond-yield', 'u.s.-6-month-bond-yield', 'u.s.-1-year-bond-yield', 'u.s.-2-year-bond-yield', 'u.s.-3-year-bond-yield', 'u.s.-5-year-bond-yield', 'u.s.-7-year-bond-yield', 'u.s.-10-year-bond-yield', 'us-20-year-bond-yield', 'u.s.-30-year-bond-yield', 'japan-1-month', 'japan-3-month-bond-yield', 'japan-6-month-bond-yield', 'japan-9-month', 'japan-1-year-bond-yield', 'japan-2-year-bond-yield', 'japan-3-year-bond-yield', 'japan-4-year-bond-yield', 'japan-5-year-bond-yield', 'japan-6-year-bond-yield', 'japan-7-year-bond-yield', 'japan-8-year-bond-yield', 'japan-9-year-bond-yield', 'japan-10-year-bond-yield', 'japan-15-year-bond-yield', 'japan-20-year-bond-yield', 'japan-30

In [142]:
btn = driver.find_element(By.XPATH, calender_new_btn)
driver.execute_script("arguments[0].click();", btn)